In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [2]:
# 导入依赖库

import pandas as pd
import glob
import os
import re
import shutil
from tqdm import trange

In [3]:
def remove_spaces(x):
    return x.replace(" ", "")

def load_raw_data(raw_data_path, output_dir):
    # 获取data_path目录下所有csv文件的路径
    file_paths = glob.glob(raw_data_path + '/*.csv')
    
    num_files = len(file_paths)
    
    # 初始化一个空DataFrame来存储所有数据
    all_data = pd.DataFrame()
    
    # 使用trange函数创建一个进度条，显示处理文件的进度
    for i in trange(num_files, desc='Processing files'):
        file_path = file_paths[i]
        file_path = file_path.replace(" ", "") 
        # 获取目录中所有CSV文件的文件名转换为日期时间格式
        date_match = re.search(r'\d{8}', file_path)
        if date_match:
            date_part = date_match.group(0)
            try:
                date = pd.to_datetime(date_part, format='%Y%m%d').date()                
                df = pd.read_csv(file_path, encoding='gbk', low_memory=False)
                
                df['合约代码'] = df['合约代码'].apply(remove_spaces)
                df['K'] = df['合约代码'].apply(lambda x: x.split('-')[-1].strip())

                df['date'] = date
                all_data = pd.concat([all_data, df], ignore_index=True)
            except ValueError:
                raise ValueError("Invalid date format")
        else:
            raise ValueError("Date or Strike not found in file path")


        
    # 假设合约代码列名为'合约代码'
    grouped = all_data.groupby('合约代码')

    # 初始化一个空的字典来存储每个合约代码对应的DataFrame
    contract_dfs = {}

    # 循环遍历每个合约代码及对应的数据，并存储为一个DataFrame
    for idx, (contract_code, group_df) in enumerate(grouped):
        contract_dfs[contract_code] = group_df

    os.makedirs(output_dir, exist_ok=True)

    progress_counter = 0
    num_contracts = len(grouped)
    # 循环遍历每个合约代码及对应的DataFrame，并将其存储为CSV文件
    for idx, (contract_code, group_df) in enumerate(grouped):
        
        file_path = os.path.join(output_dir, f'{contract_code}.csv')

        group_df = group_df.ffill()
        group_df.to_csv(file_path, index=False)

        # 更新进度计数器
        progress_counter += 1

        # 每完成一百个文件显示一次进度
        if progress_counter % 500 == 0:
            print(f"Saved {progress_counter}/{num_contracts} files")


In [4]:
# 转化原始数据

raw_data_path="2021-2024.3"
output_dir = 'available_data_path'
# load_raw_data(raw_data_path, output_dir)

if os.path.exists(output_dir):
    print(f"The directory '{output_dir}' exists.")
else:
    print(f"The directory '{output_dir}' does not exist.\nLoading...")
    load_raw_data(raw_data_path, output_dir)
    


The directory 'available_data_path' exists.


In [5]:
def copy_files(input_dir, output_dir, pattern, sub_dir):
    # 获取符合模式的文件列表
    file_paths = glob.glob(os.path.join(input_dir, pattern))
    
    # 创建子目录
    sub_dir_path = os.path.join(output_dir, sub_dir)
    os.makedirs(sub_dir_path, exist_ok=True)
    
    # 复制文件到子目录
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        dest_path = os.path.join(sub_dir_path, file_name)
        shutil.copy(file_path, dest_path)


In [6]:
# 复制文件，生成数据集
dataset_dir="datasets"

# 定义模式和子目录的对应关系
patterns = {
    '*-C-*.csv': 'Call',
    '*-P-*.csv': 'Put',
    '*HO*.csv': 'HO',
    '*IO*.csv': 'IO',
    '*MO*.csv': 'MO',
    '*IC*.csv': 'IC',
    '*IF*.csv': 'IF',
    '*IH*.csv': 'IH',
    '*IM*.csv': 'IM',
    '*TF*.csv': 'TF',
    '*TL*.csv': 'TL',
    '*TS*.csv': 'TS',
    '*T*.csv': 'T'
}

# copy_files(output_dir, dataset_dir, pattern, sub_dir)
if os.path.exists(dataset_dir):
    print(f"The directory '{dataset_dir}' exists.")
else:
    print(f"The directory '{dataset_dir}' does not exist.\nLoading...")
    for pattern, sub_dir in patterns.items():
        copy_files(output_dir, dataset_dir, pattern, sub_dir)
        


The directory 'datasets' exists.
